<a href="https://colab.research.google.com/github/RationalTangle/compose_symphony_parser/blob/main/Harmony_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Install dependencies
!rm -rf compose_symphony_parser
!git clone https://github.com/RationalTangle/compose_symphony_parser.git
!conda env create -f compose_symphony_parser/environment.yml
!pip install -r compose_symphony_parser/requirements.txt > /dev/null

In [ ]:
#@title Inputs
symphony_id = "n8Kiy3tje3CfH25500uO" #@param {type:"string"}
benchmark_ticker = "SPY" #@param {type:"string"}
use_simulated_etf_dataset = True #@param {type:"boolean"}

print(symphony_id, benchmark_ticker, use_simulated_etf_dataset)

In [ ]:
#@title Fetch Data and Compute Allocations
from compose_symphony_parser.lib import symphony_object

symphony = symphony_object.get_symphony(symphony_id)
symphony_name = symphony['fields']['name']['stringValue']
print(symphony_name)
print(f"https://app.composer.trade/symphony/{symphony_id}")

root_node = symphony_object.extract_root_node_from_symphony_response(symphony)

from compose_symphony_parser.lib import traversers, get_backtest_data

tickers = traversers.collect_referenced_assets(root_node)

#
# Get Data
#
import os
try:
  os.mkdir("data")
except FileExistsError:
  pass

closes = get_backtest_data.get_backtest_data(
    tickers.union([benchmark_ticker]), use_simulated_etf_dataset)

#
# Execute logic
#
from compose_symphony_parser.lib import transpilers
allocations, branch_tracker = transpilers.VectorBTTranspiler.execute(
    root_node, closes)

#
# Allocation / Branch Reporting
#
logic_start = branch_tracker.index.min().date()
allocations_possible_start = allocations.index.min().date()

backtest_start = allocations.dropna().index.min().date()
backtest_end = allocations.index.max().date()

print(
    f"Logic can execute from {logic_start} ({len(branch_tracker.index)})")
print(
    f"Allocations can start {allocations_possible_start} ({len(allocations.index)})")
print(f"Start: {backtest_start}")
print(f"End: {backtest_end}")
print()

allocations_aligned = allocations[allocations.index.date >= backtest_start]
branch_tracker_aligned = branch_tracker[branch_tracker.index.date >= backtest_start]

assert len(allocations_aligned) == len(branch_tracker_aligned)

In [ ]:
#@title Date Limitations
for day, ticker in sorted([(closes[ticker].dropna().index.min().date(), ticker) for ticker in closes.columns], reverse=True):
  print(f"{ticker:<5} {day}")


# Branch Reporting

In [ ]:
#@title Sanity Checks
branches_by_path = traversers.collect_branches(root_node)
branches_by_leaf_node_id = {
    key.split("/")[-1]: value for key, value in branches_by_path.items()}

# Any days without full allocation?
# Branches involved in days where allocations fail to sum to 1
if len(allocations_aligned[(allocations_aligned.sum(axis=1) - 1).abs() > 0.0001].index):
    print(allocations_aligned[(allocations_aligned.sum(axis=1) - 1).abs() > 0.0001])
    branches_by_failed_allocation_days = branch_tracker_aligned[(
        allocations_aligned.sum(axis=1) - 1).abs() > 0.0001].sum(axis=0)
    branches_with_failed_allocation_days = branches_by_failed_allocation_days[
        branches_by_failed_allocation_days != 0].index.values

    for branch_id in branches_with_failed_allocation_days:
        print(f"  -> id={branch_id} {branches_by_leaf_node_id[branch_id]}")
        print(allocations_aligned[branch_tracker_aligned[branch_id] == 1])

    raise Exception(f"Branches are misbehaving, please contact developer and provide the symphony ({symphony_id})")
else:
    print("All days are 100% allocated.")

In [ ]:
#@title Ticker by days (weighted)
ticker_allocation_weights = allocations.mean(
    axis=0).sort_values(ascending=False)
for ticker in ticker_allocation_weights.index:
    print(f"{ticker:<5} {ticker_allocation_weights[ticker]:>5.1%}")

In [ ]:
#@title Branch by days activated
backtest_days_count = len(branch_tracker)
branch_enablement = branch_tracker.mean(
    axis=0).sort_values(ascending=False)
for branch_id in branch_enablement.index:
    print(f"{branch_enablement[branch_id]:>5.1%} ({branch_enablement[branch_id] * backtest_days_count:>4.0f} of {backtest_days_count})",
          branches_by_leaf_node_id[branch_id])

# Transpile Outputs

In [ ]:
#@title Human format
from compose_symphony_parser.lib import transpilers

root_node = symphony_object.extract_root_node_from_symphony_response(symphony)
print(transpilers.HumanTextTranspiler.convert_to_string(root_node))

In [ ]:
#@title VectorBT code format
print(transpilers.VectorBTTranspiler.convert_to_string(root_node))

# VectorBT

In [ ]:
#@title Simulate Portfolio
import vectorbt as vbt

closes_aligned = closes[closes.index.date >=
                            backtest_start].reindex_like(allocations)

portfolio = vbt.Portfolio.from_orders(
    close=closes_aligned,
    size=allocations,
    size_type="targetpercent",
    group_by=True,
    cash_sharing=True,
    call_seq="auto",
    # TODO: rebalancing
    freq='D',
    # TODO: work out Alpaca fees
    fees=0,
)

In [ ]:
#@title Quantstats of VectorBT simulation
returns = portfolio.asset_returns()

# Export Quantstats HTML Report
import quantstats
import google.colab

keepcharacters = (' ', '.', '_', '-')
filepath = f"{symphony_name} - Harmony VectorBT.html"
filepath = filepath.replace("%", 'pct ')
filepath = "".join(c for c in filepath if c.isalnum() or c in keepcharacters).rstrip()
quantstats.reports.html(
    returns,
    closes[benchmark_ticker].pct_change().dropna(),
    title=f"{symphony_name} - Harmony VectorBT", output=filepath, download_filename=filepath)

google.colab.files.download(filepath)